In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import json

from PIL import Image
import cv2
import seaborn as sb

# Seb 06-01-21
import tensorflow as tf
from tensorflow import keras
import json
import math, re, os
from math import sqrt

from kaggle_datasets import KaggleDatasets
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from functools import partial

In [ ]:

!pip install --quiet /kaggle/input/kerasapplications
!pip install --quiet /kaggle/input/efficientnet-git

In [ ]:
from keras import applications

print("Tensorflow version " + tf.__version__)
print("Keras version " + keras.__version__)



In [ ]:
#Detect TPU
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
print('Number of replicas:', strategy.num_replicas_in_sync)

In [ ]:
# set up variables
AUTOTUNE = tf.data.experimental.AUTOTUNE
GCS_PATH = KaggleDatasets().get_gcs_path('cassava-leaf-disease-classification')
GCS_PATH2 = KaggleDatasets().get_gcs_path('cassava-recreate-stratificated-tfrecords')
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
IMAGE_SIZE = [512,512]
HEIGHT = 512
WIDTH = 512
CLASSES = ['0', '1', '2', '3', '4']
CHANNELS = 3
N_CLASSES = 5
EPOCHS = 25


In [ ]:
#Load the data
# split the data
#Because our data consists of training and test images only, we're going to split our training data into training and validation data using the train_test_split() function.

In [ ]:
#Decode the data
#turn the images into tensors
#normalize the image (get every pixel to have a value between 0 and 1)
def decode_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.reshape(image, [*IMAGE_SIZE, 3])
    return image

In [ ]:
#setting up variables X and y; in this case image and prediction (for images with no label)
def read_tfrecord(example, labeled):
    tfrecord_format = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "target": tf.io.FixedLenFeature([], tf.int64)
    } if labeled else {
        "image": tf.io.FixedLenFeature([], tf.string),
        "image_name": tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example['image'])
    if labeled:
        label = tf.cast(example['target'], tf.int32)
        return image, label
    idnum = example['image_name']
    return image, idnum

In [ ]:
# the following code will load the dataset using the TPU
def load_dataset(filenames, labeled=True, ordered=False):
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTOTUNE) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(partial(read_tfrecord, labeled=labeled), num_parallel_calls=AUTOTUNE)
    return dataset

In [ ]:
#note on using train_test_split()
#While I used train_test_split() to create both a training and validation dataset, consider exploring cross validation instead.

TRAINING_FILENAMES, VALID_FILENAMES = train_test_split(
    tf.io.gfile.glob(GCS_PATH2 + '/train*.tfrec'),
    test_size=0.2, random_state=5
)


TEST_FILENAMES = tf.io.gfile.glob(GCS_PATH2 + '/test_tfrecords/ld_test*.tfrec')

In [ ]:
#The following functions will be used to load our training, validation, and test datasets, as well as print out the number of images in each dataset.
def get_training_dataset():
    dataset = load_dataset(TRAINING_FILENAMES, labeled=True)  
    dataset = dataset.map(data_augment, num_parallel_calls=AUTOTUNE)  
    dataset = dataset.repeat()
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset


def get_validation_dataset(ordered=False):
    dataset = load_dataset(VALID_FILENAMES, labeled=True, ordered=ordered) 
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.cache()
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset



def get_test_dataset(ordered=False):
    dataset = load_dataset(TEST_FILENAMES, labeled=False, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset


def count_data_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)



NUM_TRAINING_IMAGES = count_data_items(TRAINING_FILENAMES)
NUM_VALIDATION_IMAGES = count_data_items(VALID_FILENAMES)
NUM_TEST_IMAGES = count_data_items(TEST_FILENAMES)

print('Dataset: {} training images, {} validation images, {} (unlabeled) test images'.format(
    NUM_TRAINING_IMAGES, NUM_VALIDATION_IMAGES, NUM_TEST_IMAGES))

In [ ]:

#adding in augmentations
def data_augment(image, label):
    # Thanks to the dataset.prefetch(AUTO) statement in the following function this happens essentially for free on TPU. 
    # Data pipeline code is executed on the "CPU" part of the TPU while the TPU itself is computing gradients.
#     image = tf.image.random_flip_left_right(image)
#     image = tf.image.random_contrast(image)
#     image = tf.image.random_jpeg_quality(image)
    p_spatial = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_rotate = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_pixel_1 = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_pixel_2 = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_pixel_3 = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_crop = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
            
    # Flips
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    if p_spatial > .75:
        image = tf.image.transpose(image)
        
    # Rotates
    if p_rotate > .75:
        image = tf.image.rot90(image, k=3) # rotate 270º
    elif p_rotate > .5:
        image = tf.image.rot90(image, k=2) # rotate 180º
    elif p_rotate > .25:
        image = tf.image.rot90(image, k=1) # rotate 90º
        
    # Pixel-level transforms
    if p_pixel_1 >= .4:
        image = tf.image.random_saturation(image, lower=.7, upper=1.3)
    if p_pixel_2 >= .4:
        image = tf.image.random_contrast(image, lower=.8, upper=1.2)
    if p_pixel_3 >= .4:
        image = tf.image.random_brightness(image, max_delta=.1)
        
#     Crops
#     if p_crop > .7:
#         if p_crop > .9:
#             image = tf.image.central_crop(image, central_fraction=.7)
#         elif p_crop > .8:
#             image = tf.image.central_crop(image, central_fraction=.8)
#         else:
#             image = tf.image.central_crop(image, central_fraction=.9)
#     elif p_crop > .4:
#         crop_size = tf.random.uniform([], int(HEIGHT*.8), HEIGHT, dtype=tf.int32)
#         image = tf.image.random_crop(image, size=[crop_size, crop_size, CHANNELS])
        
    return image, label

In [ ]:
#build model
lr_scheduler = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-1, 
    decay_steps=10000, 
    decay_rate=0.90)

In [ ]:
BATCH_SIZE = 64 * strategy.num_replicas_in_sync
EPOCHS = 75

In [ ]:
import efficientnet.keras as efn
#model
# with strategy.scope():       
#     img_adjust_layer = tf.keras.layers.Lambda(efn.preprocess_input, input_shape=[*IMAGE_SIZE, 3])
#     inputs = tf.keras.layers.Input(shape=(HEIGHT, WIDTH, CHANNELS), name='input_image')

#     base_model = efn.EfficientNetB4(input_tensor = inputs, weights=None , pooling = 'avg')

#     x = tf.keras.layers.Dropout(0.5)(base_model.output)
#     outputs = tf.keras.layers.Dense(N_CLASSES, activation = 'softmax', name = 'output')(x)
#     model = tf.keras.Model(inputs = inputs, outputs = outputs)
    


#     model.compile(
#         optimizer=tf.keras.optimizers.Adam(learning_rate=lr_scheduler, epsilon=0.001),
#         loss='sparse_categorical_crossentropy',  
#         metrics=['sparse_categorical_accuracy'])
    
#     model.summary()


In [ ]:
def model_fn(input_shape, N_CLASSES):
    inputs = tf.keras.layers.Input(shape=(input_shape), name='input_image')
    base_model = efn.EfficientNetB3(input_tensor=inputs, 
                                    include_top=False, 
                                    weights=None, 
                                    pooling='avg')
    

    x = tf.keras.layers.Dropout(.5)(base_model.output)
    output = tf.keras.layers.Dense(N_CLASSES, activation='softmax', name='output')(x)
    model = tf.keras.Model(inputs=inputs, outputs=output)

    return model

with strategy.scope():
    model = model_fn((None, None, CHANNELS), N_CLASSES)
    
model.summary()

In [ ]:
# def model_fn(input_shape, N_CLASSES):
#     inputs = L.Input(shape=input_shape, name='input_image')
#     base_model = efn.EfficientNetB(input_tensor=inputs, 
#                                     weights='imagenet', 
#                                     pooling='avg')

# #     x = L.Dropout(.5)(base_model.output)
#     output = L.Dense(N_CLASSES, activation='softmax', name='output')
#     model = Model(inputs=inputs, outputs=output)

#     return model

# with strategy.scope():
#     model = model_fn((None, None, CHANNELS), N_CLASSES)
    
# model.summary()

In [ ]:
# load data
train_dataset = get_training_dataset()
valid_dataset = get_validation_dataset()


model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr_scheduler, epsilon=0.001),
        loss='sparse_categorical_crossentropy',  
        metrics=['sparse_categorical_accuracy'])

 train the model

STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE
VALID_STEPS = NUM_VALIDATION_IMAGES // BATCH_SIZE

history = model.fit(train_dataset, 
                    steps_per_epoch=STEPS_PER_EPOCH, 
                    epochs=EPOCHS,
                    validation_data=valid_dataset,
                    validation_steps=VALID_STEPS)

In [ ]:
 #train the model

STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE
VALID_STEPS = NUM_VALIDATION_IMAGES // BATCH_SIZE

history = model.fit(train_dataset, 
                    steps_per_epoch=STEPS_PER_EPOCH, 
                    epochs=EPOCHS,
                    validation_data=valid_dataset,
                    validation_steps=VALID_STEPS)

In [ ]:
#Evaluating our model
# print out variables available to us
print(history.history.keys())


# create learning curves to evaluate model performance
history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['sparse_categorical_accuracy', 'val_sparse_categorical_accuracy']].plot();